# Aufgabe 1: SVD & PCA

> Gegeben ist eine Datenmatrix $\mathbf{X}\in\mathbb{R}^{1000\times30}$ mit
> Singulärwerten
> 
> $$\sigma_j = \begin{cases} 2^{8-j} &,~j=1,\ldots,20 \\ 0 &,~j>20\end{cases}$$

In [1]:
import numpy as np
np.set_printoptions(precision=4, suppress=True)

n = 1_000
m = 30

def sigma(j):
    if j > 20:
        return 0
    return 2 ** (8 - j)

In [2]:
sigmas = []
for j in range(1, m + 1):
    sigmas.append(sigma(j))
sigmas = np.array(sigmas)
print(sigmas)

[128.      64.      32.      16.       8.       4.       2.       1.
   0.5      0.25     0.125    0.0625   0.0312   0.0156   0.0078   0.0039
   0.002    0.001    0.0005   0.0002   0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.    ]


## 2- und Frobenius-Norm

> Berechnen Sie $||\mathbf{X}||_2, ||\mathbf{X}||_F$

$$
\sigma_1 \ge \sigma_2 \ge \ldots \ge \sigma_m \ge 0 \newline
\Rightarrow ||\mathbf{X}||_2 = \max \sigma_j = \sigma_1
$$

$$||\mathbf{X}||_F = \sqrt{ \sum_{j=1}^\ell \sigma_j }$$

In [3]:
X_2 = sigma(1)
# X_2 = np.max(sigmas)
print(f'X_2 = {X_2}')

X_2 = 128


In [4]:
X_F = np.sum(sigmas ** 2)
print(f'X_F = sqrt({X_F:.4f})', end=' ')
X_F = np.sqrt(X_F)
print(f'= {X_F:.4f}')

X_F = sqrt(21845.3333) = 147.8017


## Effektive Konditionszahl $\kappa_2(\mathbf{X})$

> Berechnen Sie die effektive Konditionszahl
> $\kappa_2(\mathbf{X}) = \frac{\max \sigma_j}{\min \sigma_j}$ mit
> $\sigma_j\ne0$

In [5]:
sigmas_non_zero = sigmas[np.nonzero(sigmas)]

cond2_X = np.max(sigmas_non_zero) / np.min(sigmas_non_zero)
print(f'cond2(X) = {cond2_X}')

cond2(X) = 524288.0


## Exakter Rang

> Berechnen Sie den exakten Rang von $\mathbf{X}$.

$\text{rang}(\mathbf{X}) = |\sigma_j \ne 0|$

In [6]:
print(sigmas_non_zero.shape, '==>', sigmas_non_zero.shape[0])

(20,) ==> 20


## Numerischer Rang

> Berechnen Sie den numerischen Rang von $\mathbf{X}$ in Singe Precision
> (Maschinen-Genauigkeit $\texttt{eps} = 1.19209 \cdot 10^{-7}$).

Numerischer Rang ($\varepsilon$-Rang) von $\mathbf{X}\in\mathbb{R}^{n\times m}$
mit Singulärwerten $\left( \sigma_j\right)_{j=1}^m$ und Toleranz
$0<\varepsilon \ll n:$

$\operatorname{rang}_\varepsilon(\mathbf{X}) = |\sigma_j>\varepsilon|$

mit $\varepsilon = \max{(n,m)} \cdot \Vert \mathbf{X} \Vert_2 \cdot \texttt{eps}$

In [7]:
eps = 1.19209 * 10**(-7)
val = X_2 * n * eps # sigma(1) * n * eps
print(f'val = {val:.4f}')

rank_eps = np.count_nonzero(sigmas > val)
print(f'rank_ε = {rank_eps}')

# import sympy
# import math

# j = sympy.Symbol('j')
# j = sympy.solve(2 ** (8 - j) > val)
# j = math.floor(j.rhs)
# print(f'rank_ε = {j}')

val = 0.0153
rank_ε = 14


## Relativer Fehler in 2-Norm der Rang-5 Best-Approximation

> Berechnen Sie den relativen Fehler in der 2-Matrixnorm
> $(||\mathbf{X} - \mathbf{X}_r||_2 / ||\mathbf{X}||_2)$ einer Rang-5
> Best-Approximation, sowie die einhergehenden Speichereinsparungen in Single
> Precision.

$\displaystyle
\text{relativer Fehler: } \frac{||\mathbf{X} - \mathbf{X}_k||_2}{||\mathbf{X}||_2}
= \frac{\sigma_{k+1}}{\sigma_1}
$

$\text{Speicher}=\#\text{Zeilen} \cdot \# \text{Spalten} \cdot 32~\text{Bit}$

In [8]:
k = 5
print(sigma(k + 1) / sigma(1))

0.03125


In [9]:
speicher_original = n * m * 32
speicher_rang_approx = k * (n + m + 1) * 32

einsparung = (1 - speicher_rang_approx / speicher_original) * 100
print(f'{einsparung:.4f}%')

82.8167%


## Bestimmung Rang

> Berechnen Sie den notwendigen Rang $r$ einer Best-Rang-$r$-Approximation,
> damit der relative Fehler in der $2$-Matrixnorm kleiner als $10^{-1}$ ist.

$$
\frac{|| \mathbf{X} - \mathbf{X}_r ||_2}{|| \mathbf{X} ||_2}
= \frac{\sigma_{r+1}}{\sigma_m} < \text{Tol}
$$

In [10]:
tol = 10**(-1)

r = 0
sigmas_max = np.max(sigmas)
for i in range(0, len(sigmas_non_zero)):
    val = sigmas_non_zero[i] / sigmas_max
    if val < tol:
        print(f'i = {i:2}:\t{val:.4f} < {tol:.4f}')
        r = i
        break
    else:
        print(f'i = {i:2}:\t{val:.4f} ≥ {tol:.4f}')

print(f'==> r = {r}')

i =  0:	1.0000 ≥ 0.1000
i =  1:	0.5000 ≥ 0.1000
i =  2:	0.2500 ≥ 0.1000
i =  3:	0.1250 ≥ 0.1000
i =  4:	0.0625 < 0.1000
==> r = 4


## Totale Varianz

> Nehmen wir nun an, dass $\mathbf{X}$ zentrierte Daten enthält.
>
> Berechnen Sie die totale Varianz.

$$T = \frac{1}{n-1} ||\mathbf{X}||_F^2$$

In [11]:
T = 1 / (n - 1) * X_F**2
print(f'T = {T:.4f}')

T = 21.8672


## 80% Varianz Hauptkomponenten

> In wie vielen Hauptkomponenten liegen 80% der Totalen Varianz?

$$
\frac{\frac{1}{n - 1} \sum_{j=1}^{{\color{red} q} \le \ell} \sigma_j^2}{T} 
= \frac{\sum_{j=1}^{{\color{red} q}}}{\sum_{j=1}^{\ell} \sigma_j^2}
\le \operatorname{Tol}
$$

In [12]:
tol = 0.8

# konstant --> nur 1x berechnen
denominator = np.sum(sigmas ** 2)

for q in range(1, m):
    numerator = np.sum(sigmas[:q] ** 2)
    val = numerator / denominator
    if val > tol:
        print(f'q = {q}:\t{val:.4f} > {tol:.4f}')
        print(f'==> q = {q}')
        break
    else:
        print(f'q = {q}:\t{val:.4f} ≤ {tol:.4f}')

q = 1:	0.7500 ≤ 0.8000
q = 2:	0.9375 > 0.8000
==> q = 2
